In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
from tensorflow.keras.models import load_model

# Define class labels (update as per your model training)
class_labels = ["cat", "dog", "elephant", "horse", "lion"]  # Example classes

# Path to the full saved model
full_model_path = "model_animal.weights.h5"  # Update this to the actual file path

# Load the full saved model
try:
    model = load_model(full_model_path)
    print("Full model loaded successfully!")
except Exception as e:
    print("Error loading model:", e)
    exit()

# List of carnivorous animals
carnivorous_animals = ["cat", "dog", "lion"]

# Function to detect animals in an image
def detect_animals(image_path):
    # Load image
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width = img.shape[:2]

    # Resize image to match the model's expected input size (224x224)
    img_resized = cv2.resize(img_rgb, (224, 224))

    # Preprocess the image: Normalize and expand dimensions
    img_normalized = img_resized / 255.0
    img_expanded = np.expand_dims(img_normalized, axis=0)  # Add batch dimension

    # Get model predictions
    predictions = model.predict(img_expanded)

    # Get the predicted class (with the highest probability)
    class_idx = np.argmax(predictions)
    class_label = class_labels[class_idx]
    confidence = predictions[0][class_idx]

    # Draw bounding box and label on the image if a carnivorous animal is detected
    carnivores_count = 0
    if class_label in carnivorous_animals and confidence > 0.5:  # Only consider if confidence is high
        carnivores_count += 1
        color = (0, 0, 255)  # Red for carnivorous animals
    else:
        color = (0, 255, 0)  # Green for others

    # Draw bounding box and label
    text = f"{class_label}: {confidence:.2f}"
    cv2.putText(img, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
    return img, carnivores_count

# Function to open and detect animals in an image
def open_image():
    # File dialog to choose an image
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if not image_path:
        return

    img, carnivores_count = detect_animals(image_path)

    # Display a pop-up message with carnivorous count
    messagebox.showinfo("Carnivorous Animals Detected", f"Number of carnivorous animals: {carnivores_count}")

    # Convert BGR image to RGB for displaying in Tkinter
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    img = ImageTk.PhotoImage(img)

    # Display the image in the GUI
    preview_label.config(image=img)
    preview_label.image = img

# Function to open and detect animals in a video
def open_video():
    # File dialog to choose a video
    video_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mkv")])
    if not video_path:
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        messagebox.showerror("Error", "Could not open video file")
        return

    carnivores_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame, carnivores_count_frame = detect_animals(frame)
        carnivores_count += carnivores_count_frame

        # Display the frame in a window
        cv2.imshow("Video Detection", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Display a pop-up message with carnivorous count
    messagebox.showinfo("Carnivorous Animals Detected", f"Number of carnivorous animals: {carnivores_count}")

# Create Tkinter GUI
root = tk.Tk()
root.title("Animal Detection Model")

# Buttons for image and video detection
image_button = tk.Button(root, text="Open Image", command=open_image, width=20, height=2)
image_button.pack(pady=10)

video_button = tk.Button(root, text="Open Video", command=open_video, width=20, height=2)
video_button.pack(pady=10)

# Label for preview
preview_label = tk.Label(root)
preview_label.pack(pady=10)

# Start the GUI
root.mainloop()
